# Preprocesados

Aplicar operaciones sobre los datos con el fin de mejorar los modelados

* Escalado de datos
    * StandardScaler
    * MinMaxScaler
    * RobustScaler
* Transformación de distribuciones de datos (intentar reducir la asimetría de los datos) (similar a aplicar función raíz o logaritmo a los datos)
    * QuantileTransformer
    * PowerTransformer
* Encoders para codificación de categóricos a numéricos:
    * OneHotEncoder (Equivalente a pd.get_dummies) Habitual usarlo en la entrada X.
    * LabelEncoder (Equivalente a hacer un .map() en pandas con un diccionario). Habitual usarlo en salida y.

* Imputers:
    * SimpleImputer: mean, median, most_frequentKNNImputerIterativeImpute
    * KNNImputer
    * IterativeImputer


Todas estas clases tienen algo en común, tienen métodos fit y transform para que puedan usarse de forma similar, lo que cambia es las operaciones que realizan sobre los datos, por ejemplo: escalar, transformar, codificar, imputar, discretizar, binarizar, normalizar, estandarizar...

Los pipelines de scikit learn simplifican el uso de preprocesadores cuando queremos aplicar varios de ellos y combinarlos.


In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


In [2]:
df = sns.load_dataset('diamonds').dropna().sample(5000, random_state=42).reset_index(drop=True) # dropna borra los nulos
df.head(3)


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.24,Ideal,G,VVS1,62.1,56.0,559,3.97,4.00,2.47
1,0.58,Very Good,F,VVS2,60.0,57.0,2201,5.44,5.42,3.26
2,0.40,Ideal,E,VVS2,62.1,55.0,1238,4.76,4.74,2.95


* Particionar y crear método calculate_metrics para hacer un modelado antes de hacer nada y ver si aplicando prepprocesados mejora

In [3]:
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error,mean_absolute_percentage_error

X = df[['carat', 'depth', 'table','x','y','z']]
y = df['price']

X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

df_resultados = pd.DataFrame(columns=['Modelo', 'Preprocesados', 'R2', 'MAE','RMSE','MAPE'])

def calculate_metrics(prepreocesados_name, x_train, x_test, y_train, y_test):
    models = {
        'LinearRegression': LinearRegression(),
        'KNN': KNeighborsRegressor(),
        'SVR': SVR(),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'RandomForest': RandomForestRegressor(random_state=42)
        
    }
    
    for model_name, model in models.items():
        model.fit(x_train,y_train)
        y_pred = model.predict(x_test)
        df_resultados.loc[len(df_resultados)] = [model_name, prepreocesados_name,
                                                r2_score(y_test, y_pred), 
                                                 mean_absolute_error(y_test, y_pred),
                                                 root_mean_squared_error(y_test, y_pred),
                                                 mean_absolute_percentage_error(y_test, y_pred)
                                ]
    return df_resultados.sort_values('R2', ascending=False)
        
        

### Nos da el resultado sin procesar y vemos que el Random Forest de momento es la mejor, ahora llega el momento de intentar mejorar los datos para subir ese rango

In [4]:
calculate_metrics('Sin preprocesados', X_train,x_test,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
1,KNN,Sin preprocesados,0.849006,899.186200,1604.254748,0.226420
3,DecisionTree,Sin preprocesados,0.766569,1122.743000,1994.676075,0.274400
2,SVR,Sin preprocesados,-0.179999,2967.146616,4484.706331,1.100950


# StandardScaler
 * Cuándo usarlo:
  - * Cuando los datos no tienen outliers extremadamente grandes 
  - * Es el escaladomás común, especialmente para algoritmos que asumen normalidad o que son sensibles a la escala 
  Transforma los datos para que cada característica tenga media 0 y desviación estándar 1.

$$
X_{\text{scaled}} = \frac{X - \mu}{\sigma}
$$

donde $\mu$ es la media y $\sigma$ la desviación estándar (calculados **solamente** en el conjunto de entrenamiento).

**Cuándo usarlo**:
- Cuando los datos no tienen outliers extremadamente grandes (o son relativamente cercanos a una distribución normal).
- Es el escalado más común, especialmente para algoritmos que asumen normalidad o que son sensibles a la escala (regresiones lineales, redes neuronales, SVM, etc.).


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train) # fit solo sobre train y no sobre test para evitar data leakage

x_train_scaled = scaler.transform(X_train) # devuelve un array de numpy
x_test_scaled = scaler.transform(x_test)

# Opcional, pasarlo aun DataFrame de pandas con los nombres de las columnas
x_train_scaled = pd.DataFrame(x_train_scaled, columns=X.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=X.columns)
x_train_scaled.head(3)

,carat,depth,table,x,y,z
0,-1.028331,0.973355,-0.630227,-1.277261,-1.259030,-1.173678
1,0.439665,2.353854,0.692667,0.507249,0.500558,0.810866
2,0.104123,-0.338119,0.251703,0.347442,0.295124,0.278778


In [6]:
calculate_metrics('StandardScaler', x_train_scaled,x_test_scaled,y_train, y_test) # Este scaler no nos ha mejorado los datos 

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
1,KNN,Sin preprocesados,0.849006,899.186200,1604.254748,0.226420
8,DecisionTree,StandardScaler,0.766985,1119.155000,1992.897756,0.273665
3,DecisionTree,Sin preprocesados,0.766569,1122.743000,1994.676075,0.274400
7,SVR,StandardScaler,0.033553,2363.397416,4058.655409,0.675242
2,SVR,Sin preprocesados,-0.179999,2967.146616,4484.706331,1.100950


# MinMaxScaler

**Cuándo usarlo**:
   * Cuando quieres que los datos estén acotados entre **0 y 1**
   * Sin embargo es muy sensible a los outliers

Escala y traslada cada característica individual a un rango definido, por defecto $[0,1]$.

$$
X_{\text{scaled}} = \frac{X - X_{\min}}{X_{\max} - X_{\min}}
$$


**Cuándo usarlo**:

- Cuando quieres que los datos estén **acotados entre 0 y 1** o entre otro rango definido (por ejemplo, $[-1, 1]$), porque se puede personalizar el rango a $[min, max]$. Para algoritmos basados en distancias como KNN.
- Sin embargo, **es muy sensible a los outliers**. Un valor muy grande puede comprimir el resto de datos.


In [7]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(x_test)

x_train_scaled = pd.DataFrame(x_train_scaled, columns=X.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=X.columns)
x_train_scaled.head(3)

# El resultado elimina los datos negativos


,carat,depth,table,x,y,z
0,0.026247,0.50000,0.291667,0.066773,0.077901,0.265306
1,0.209974,0.62500,0.416667,0.386328,0.391097,0.546939
2,0.167979,0.38125,0.375000,0.357711,0.354531,0.471429


In [8]:
calculate_metrics('MinMaxScaler', x_train_scaled,x_test_scaled,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
11,KNN,MinMaxScaler,0.857112,868.646400,1560.600343,0.220208
1,KNN,Sin preprocesados,0.849006,899.186200,1604.254748,0.226420
8,DecisionTree,StandardScaler,0.766985,1119.155000,1992.897756,0.273665


# RobustScaler

* Cuándo lo usamos:
* Cuando existen outliers en los datos y nos pueden afectar

Escala los datos usando **mediana** e **IQR** (rango intercuartílico).

$$
X_{\text{scaled}} = \frac{X - \text{mediana}(X)}{\text{IQR}}
$$
donde $\text{IQR} = Q_3 - Q_1$.

**Cuándo usarlo**:

Cuando existen **outliers** en los datos que pueden afectar mucho al escalado.
Al usar la mediana y el IQR en lugar de la media y desviación estándar, resulta mucho **menos sensible a valores atípicos**.




In [9]:
scaler = RobustScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(x_test)

# Lo pasamos aún DataFrame para ver más claro todo en columnas 
x_train_scaled = pd.DataFrame(x_train_scaled, columns=X.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=X.columns)
x_train_scaled.head(3)

,carat,depth,table,x,y,z
0,-0.62500,0.866667,-0.333333,-0.768176,-0.767956,-0.707965
1,0.46875,2.200000,0.666667,0.334705,0.320442,0.513274
2,0.21875,-0.400000,0.333333,0.235940,0.193370,0.185841


In [13]:
calculate_metrics('RobustScaler', x_train_scaled,x_test_scaled,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
19,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
24,RandomForest,RobustScaler,0.868256,835.223960,1498.505051,0.210161
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
15,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
20,LinearRegression,RobustScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
21,KNN,RobustScaler,0.858300,876.919000,1554.096430,0.221305


# Quantile Transformer

Es una transformación basada en **cuantiles**:

1. Ordena los valores de cada columna y les asigna su posición cuantílica (e.g. percentiles).

2. Mapea esos cuantiles ya sea a una distribución **uniforme** en $[0,1]$ o a una distribución **normal** (Gaussiana) si se especifica `output_distribution='normal'`.

- Por defecto, `output_distribution='uniform'`, lo que hace que cada característica se distribuya aproximadamente **de manera uniforme** en $[0, 1]$.

- Si pones `output_distribution='normal'`, intentará que los datos se parezcan a una **distribución normal (Gaussiana)** con media 0 y desviación estándar 1.

¿Cuándo usarlo?

- Cuando quieres aplanar la distribución de una variable que está muy sesgada (skewed) o con colas largas. El método de cuantiles “estira” y “comprime” la distribución de forma que cada cuantil se mapea a un cuantil de la distribución objetivo (uniforme o normal).

- Es útil cuando quieres datos:
  - Bien distribuidos entre $[0, 1]$ (caso uniforme).
  - O aproximar una Gaussiana sin realizar transformaciones paramétricas (e.g. logaritmo).

Puede ser más fuerte que raíz o logaritmo porque no solo reduce sesgo redistribuye los valores, puede ser demasiado agresivo si los datos ya son simétricos.



In [10]:
X.skew() # skew superior a 0 o inferior a 0 es que está SESGADO

carat    1.160939
depth    0.085048
table    0.712548
x        0.425650
y        0.424264
z        0.407379
dtype: float64

In [11]:
from sklearn.preprocessing import QuantileTransformer


transformer = QuantileTransformer()

x_train_transformed = transformer.fit_transform(X_train)
x_test_transformed = transformer.transform(x_test)

# Lo pasamos aún DataFrame para ver más claro todo en columnas 
x_train_transformed = pd.DataFrame(x_train_transformed, columns=X.columns)
x_test_transformed = pd.DataFrame(x_test_transformed, columns=X.columns)
x_train_transformed.head(3)

,carat,depth,table,x,y,z
0,0.059560,0.883383,0.285285,0.041041,0.057558,0.131632
1,0.670671,0.983984,0.773273,0.657157,0.657157,0.787788
2,0.598599,0.307307,0.629129,0.611612,0.602603,0.595596


In [12]:
x_train_transformed.skew() # Vemos se ha reducido la asimetria los sesgos

carat    0.001497
depth   -0.000490
table    0.007868
x       -0.000119
y        0.000009
z        0.000117
dtype: float64

In [13]:
calculate_metrics('QuantileTransformer', x_train_transformed,x_test_transformed,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
11,KNN,MinMaxScaler,0.857112,868.646400,1560.600343,0.220208
1,KNN,Sin preprocesados,0.849006,899.186200,1604.254748,0.226420


# PowerTransformer
* Transforman la distribuciones de los datos

intenta que los datos sean parecidos a una distribución normal.

PowerTransformer aplica transformaciones de **potencia** para hacer que los datos se acerquen más a una distribución normal.

- Admite dos métodos principales:

  1. **Box-Cox**: requiere que todos los datos sean **estrictamente positivos**.
  2. **Yeo-Johnson**: puede manejar datos con valores 0 o negativos.  Yeo-Johnson es una versión mejorada de Box-Cox que funciona con valores negativos y positivos.

Internamente, `PowerTransformer` encuentra el mejor parámetro de potencia que estabiliza la varianza y reduce la asimetría (skew) de los datos, por tanto es una opción más flexible y automatizada que aplicar manualmente un np.sqrt o np.log a una columna.

¿Cuándo usarlo?

- Cuando tus datos están fuertemente sesgados (tienen heavy skew) y necesitas **mejorar la normalidad**. El QuantileTransfomer podría ser más fuerte.

- Se suele usar antes de **modelos lineales** o algoritmos que asumen distribuciones aproximadamente gaussianas, ayudando a cumplir hipótesis de homocedasticidad (misma varianza) y mejorando la linealidad.

- Si tus datos tienen valores cero o negativos, no puedes usar Box-Cox, pero sí Yeo-Johnson.
- Puedes luego aplicar un escalado adicional (por ejemplo, `StandardScaler`) tras la transformación de potencia si lo deseas.


In [14]:
from sklearn.preprocessing import PowerTransformer

transformer = PowerTransformer()

x_train_transformed = transformer.fit_transform(X_train)
x_test_transformed = transformer.transform(x_test)

# Lo pasamos aún DataFrame para ver más claro todo en columnas 
x_train_transformed = pd.DataFrame(x_train_transformed, columns=X.columns)
x_test_transformed = pd.DataFrame(x_test_transformed, columns=X.columns)
x_train_transformed.head(3)

,carat,depth,table,x,y,z
0,-1.332196,0.974480,-0.592305,-1.430225,-1.403233,-1.264652
1,0.715544,2.309622,0.764271,0.613714,0.607868,0.866351
2,0.404235,-0.329079,0.350714,0.469433,0.420965,0.393914


In [15]:
print('skew antes', x_train_transformed.skew())
print('\nskew después: \n', x_train_transformed.skew())

skew antes carat    0.127325
depth    0.002243
table   -0.005639
x        0.037489
y        0.037876
z        0.030497
dtype: float64

skew después: 
 carat    0.127325
depth    0.002243
table   -0.005639
x        0.037489
y        0.037876
z        0.030497
dtype: float64


In [16]:
calculate_metrics('PowerTransformer', x_train_transformed,x_test_transformed,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
24,RandomForest,PowerTransformer,0.868020,834.399099,1499.846193,0.209508
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
11,KNN,MinMaxScaler,0.857112,868.646400,1560.600343,0.220208


In [17]:
transformer = PowerTransformer(standardize=False)

x_train_transformed = transformer.fit_transform(X_train)
x_test_transformed = transformer.transform(x_test)

# Lo pasamos aún DataFrame para ver más claro todo en columnas 
x_train_transformed = pd.DataFrame(x_train_transformed, columns=X.columns)
x_test_transformed = pd.DataFrame(x_test_transformed, columns=X.columns)
x_train_transformed.head(3)
calculate_metrics('PowerTransformer standar False', x_train_transformed,x_test_transformed,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
24,RandomForest,PowerTransformer,0.868020,834.399099,1499.846193,0.209508
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
29,RandomForest,PowerTransformer standar False,0.862704,849.270065,1529.755989,0.213348
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesados,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851


# OneHotEncoder

* Equivalente a get_dummies de pandas pero es de scikit Learn
* Se suelen usar en las entradas 
* Tiene un parámetro sparse_output: Que devuelve una matriz densa o matriz dispersa
* Si queremos ponerlo en un DataFrame deberemos usar el sparse_output
* OneHotEncoder crea columnas

**Cuándo usarlo**:

1. Para **features categóricas nominales** (sin orden), como color, ciudad, tipo de mascota, etc.
2. Normalmente se aplica a **variables de entrada** (X).
3. Útil en la mayoría de los modelos que necesitan variables numéricas y no tienen forma de manejar directamente categorías.
4. Se puede usar en pipelines de scikit learn

Parámetro sparse_output:

* sparse_output=True: Devuelve la transformación como una matriz dispersa (scipy.sparse.csr_matrix) en lugar de un numpy.ndarray.
    * Ventaja: Usa menos memoria si hay muchas categorías con muchos ceros (matriz dispersa)
    * Desventaja: Puede ser incompatible con algunas funciones de Pandas y Scikit-learn que esperan una matriz densa.
* sparse_output=False: Devuelve la transformación como un array denso (numpy.ndarray), en lugar de una matriz dispersa.
    * Ventaja: Se puede convertir fácilmente en un DataFrame de Pandas sin errores ni conversiones adicionales.
    * Desventaja: Puede consumir más memoria si hay muchas categorías y muchos ceros.

* Diferencia entre matriz densa y dispersa:
    * Matriz densa: Es una matriz donde todos los valores, incluyendo los ceros, son almacenados en memoria.
    * Matriz dispersa: Es una matriz en la que se almacenan solo los valores distintos de cero, junto con sus coordenadas (índices de fila y columna). Más óptima pero más difícil de manipular directamente, requiere conversión a formato denso para ciertas operaciones.


In [18]:
X = df[['carat', 'depth', 'table','x','y','z', 'cut', 'color', 'clarity']]
y = df['price']

X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

pd.get_dummies(X)

,carat,depth,table,x,y,z,cut_Ideal,cut_Premium,cut_Very Good,cut_Good,...,color_I,color_J,clarity_IF,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
0,0.24,62.1,56.0,3.97,4.00,2.47,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,0.58,60.0,57.0,5.44,5.42,3.26,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
2,0.40,62.1,55.0,4.76,4.74,2.95,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,0.43,60.8,57.0,4.92,4.89,2.98,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
4,1.55,62.3,55.0,7.44,7.37,4.61,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.31,60.8,57.0,4.40,4.38,2.67,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4996,1.06,61.2,55.0,6.57,6.61,4.03,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4997,0.71,61.0,56.0,5.77,5.80,3.53,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4998,0.90,63.3,56.0,6.13,6.10,3.87,False,False,True,False,...,False,True,False,False,False,False,True,False,False,False


In [19]:
X_train.select_dtypes(exclude=['object', 'category']).columns.to_list() # Este proceso nos extrae las numéricas

['carat', 'depth', 'table', 'x', 'y', 'z']

In [20]:
X_train.select_dtypes(include=['object', 'category']).columns.to_list() # Este proceso nos extrae las categoricas o str

['cut', 'color', 'clarity']

In [21]:
from sklearn.preprocessing import OneHotEncoder
# separamos las columnas categoricas y las numçericas para no nos de errores 
numerical_columns = X_train.select_dtypes(exclude=['object', 'category']).columns.to_list()
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns.to_list()
encoder = OneHotEncoder(sparse_output=False) #sparse_ouput=False para obtenerlo como matriz de 0s y 1s

x_train_encoded = encoder.fit_transform(X_train[categorical_columns]) # Esto es un array de numpy con las codificaciones
x_test_encoded = encoder.transform(x_test[categorical_columns])

# Pasarlo a dataframe de pandas y juntarlo con las numéricas para obtener resultado como pd.get_dummies
#encoder.get_feature_names_out() # los nombres de las nuevas columnas que sean generado
x_train_final = pd.concat(
  [  
    pd.DataFrame(x_train_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True), # Categoricas
    X_train[numerical_columns].reset_index(drop=True) # Numéricas
  ],
   axis=1
 )   
 
x_test_final = pd.concat(
  [  
    pd.DataFrame(x_test_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True), # Categoricas
    x_test[numerical_columns].reset_index(drop=True) # Numéricas
  ],
   axis=1
 )  




In [22]:

calculate_metrics('OneHotEncoder', x_train_final,x_test_final,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
34,RandomForest,OneHotEncoder,0.963541,397.252065,788.302258,0.097110
33,DecisionTree,OneHotEncoder,0.930697,528.805000,1086.851350,0.126546
30,LinearRegression,OneHotEncoder,0.914045,791.159437,1210.397029,0.438661
31,KNN,OneHotEncoder,0.887188,764.857600,1386.662491,0.201587
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
24,RandomForest,PowerTransformer,0.868020,834.399099,1499.846193,0.209508
4,RandomForest,Sin preprocesados,0.867955,835.518392,1500.214999,0.210188
29,RandomForest,PowerTransformer standar False,0.862704,849.270065,1529.755989,0.213348


# Combinar OneHotEncoder con MInMaxScaler

In [23]:
from sklearn.preprocessing import OneHotEncoder
# separamos las columnas categoricas y las numçericas para no nos de errores 
numerical_columns = X_train.select_dtypes(exclude=['object', 'category']).columns.to_list()
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns.to_list()
encoder = OneHotEncoder(sparse_output=False) #sparse_ouput=False para obtenerlo como matriz de 0s y 1s

x_train_encoded = encoder.fit_transform(X_train[categorical_columns]) # Esto es un array de numpy con las codificaciones
x_test_encoded = encoder.transform(x_test[categorical_columns])

scaler= MinMaxScaler()
x_train_scaled = scaler.fit_transform(X_train[numerical_columns])
x_test_scaled = scaler.transform(x_test[numerical_columns])

# Pasarlo a dataframe de pandas y juntarlo con las numéricas para obtener resultado como pd.get_dummies
#encoder.get_feature_names_out() # los nombres de las nuevas columnas que sean generado
x_train_final = pd.concat(
  [  
    pd.DataFrame(x_train_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True), # Categoricas
    pd.DataFrame(x_train_scaled,columns=numerical_columns).reset_index(drop=True) # Numéricas
  ],
   axis=1
 )   
 
x_test_final = pd.concat(
  [  
    pd.DataFrame(x_test_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True), # Categoricas
    pd.DataFrame(x_test_scaled,columns=numerical_columns).reset_index(drop=True) # Numéricas
  ],
   axis=1
 )  

In [24]:
calculate_metrics('OneHotEncoder+MinMaxscaler', x_train_final,x_test_final,y_train, y_test)

,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
34,RandomForest,OneHotEncoder,0.963541,397.252065,788.302258,0.097110
39,RandomForest,OneHotEncoder+MinMaxscaler,0.963456,397.241947,789.227699,0.097156
38,DecisionTree,OneHotEncoder+MinMaxscaler,0.935301,519.908000,1050.125344,0.126376
33,DecisionTree,OneHotEncoder,0.930697,528.805000,1086.851350,0.126546
35,LinearRegression,OneHotEncoder+MinMaxscaler,0.914293,791.529000,1208.650284,0.439170
30,LinearRegression,OneHotEncoder,0.914045,791.159437,1210.397029,0.438661
31,KNN,OneHotEncoder,0.887188,764.857600,1386.662491,0.201587
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055


# LabelEncoder

Convierte etiquetas (categorías) a valores numéricos enteros de 0 a n-1.

Por ejemplo, si tienes las categorías `["rojo", "verde", "azul"]`, podría asignar
- *rojo* $\to 0$,
- *verde* $\to 1$,
- *azul* $\to 2$.

* Normalmente se usa para la variable de salida (y) si se trata de un problema de clasificación multiclase.
* Convierte cada clase categórica a un entero distinto.
* También puede usarse en columnas de entrada si (y solo si) tienen un orden real (caso ordinal) o si el modelo puede manejarlo sin suponer que 2 > 1 > 0 (pero esto no es común; en features categóricas nominales, lo típico es OneHotEncoder).

Equivalente a cuando hacemos el `df['class'].map({'setosa':0, 'virginica':1, 'versicolor':2})`


In [25]:
X = df[['carat', 'depth', 'table','x','y','z', 'price']]
y = df['cut'] # Categorica


In [26]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, x_test, y_train, y_test = train_test_split(X,y_encoded,test_size=0.20, random_state=42)

print('clases del encoder:', encoder.classes_)
print('ejemplo y_encoded:', y_encoded[:10])
print('ejemplo y_train:', y_train[:10])
print('ejemplo y_test:', y_test[:10])

clases del encoder: ['Fair' 'Good' 'Ideal' 'Premium' 'Very Good']
ejemplo y_encoded: [2 4 2 3 2 0 2 2 3 2]
ejemplo y_train: [1 0 3 4 1 2 2 2 2 4]
ejemplo y_test: [3 3 2 3 2 2 2 2 2 1]


In [27]:
# Con inverse_transform obtienes las categorias originales a parttir de los datos codificados

encoder.inverse_transform(y_encoded)[:10]

array(['Ideal', 'Very Good', 'Ideal', 'Premium', 'Ideal', 'Fair', 'Ideal',
       'Ideal', 'Premium', 'Ideal'], dtype=object)

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred= model.predict(x_test)
accuracy_score(y_test, y_pred)

0.749

In [29]:
print('predicciones', y_pred[:10])
print('predicciones descodificads',encoder.inverse_transform(y_pred)[:10])

predicciones [3 3 2 3 2 2 2 2 2 1]
predicciones descodificads ['Premium' 'Premium' 'Ideal' 'Premium' 'Ideal' 'Ideal' 'Ideal' 'Ideal'
 'Ideal' 'Good']


# KBinsDiscretizer

* Similar a pd.cut de pandas para descritizar columnas numéricas
Convierte variables numéricas continuas en variables discretas, dividiendo los valores en **intervalos o "bins"**. Cada intervalo recibe una etiqueta numérica.

¿Cuándo usarlo?

* Cuando queremos convertir variables numéricas continuas en categorías discretas (por ejemplo, dividir `carat` en "pequeño", "mediano" y "grande").
* Cuando un modelo puede beneficiarse de información categorizada en lugar de valores continuos.
* Para mejorar la interpretabilidad de un modelo.

Formas de discretización:

- `uniform`: Divide el rango en intervalos de **igual tamaño**.
- `quantile`: Crea intervalos con **igual número de muestras**.
- `kmeans`: Usa **K-Means** para definir los bins.


In [30]:
# realizar clasificación multiclase sobre price

X = df[['carat', 'depth', 'table','x','y','z']]
y = df[['price']] # es numérica pero la vamos a transformar en categorica --> clasificación multiclase

X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

# Mejor hacerlo por separado las categorias

In [31]:
from sklearn.preprocessing import KBinsDiscretizer

# convierte variables numéricas en categoricas
discretizer = KBinsDiscretizer(encode='ordinal', n_bins=4, strategy='kmeans') # n_bins lo divide(discretizar) en este caso en 4 barato,caro etc...
discretizer.fit(y_train)

y_train_descritized = discretizer.transform(y_train).ravel() # lo pasa de 2 dimensiones a una dimensión
y_test_descritized = discretizer.transform(y_test).ravel()

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train_descritized)
y_pred= model.predict(x_test)
accuracy_score(y_test_descritized, y_pred)

0.853

In [68]:
print('predictions:', y_pred[:10])

predictions: [0. 1. 0. 1. 0. 0. 0. 1. 1. 1.]


In [32]:
print('discretizer.n_bins:', discretizer.n_bins_)
print('discretizer.n_features_in:', discretizer.n_features_in_)
print('discretizer.n_features_in:', discretizer.feature_names_in_)
print('discretizer.bin_edges_:', discretizer.bin_edges_)
print('bin min', discretizer.bin_edges_[0][0])
print('bin 1', discretizer.bin_edges_[0][1])
print('bin 2', discretizer.bin_edges_[0][2])
print('bin 3', discretizer.bin_edges_[0][3])
print('bin max', discretizer.bin_edges_[0][4])

discretizer.n_bins: [4]
discretizer.n_features_in: 1
discretizer.n_features_in: ['price']
discretizer.bin_edges_: [array([  336.        ,  2964.95252869,  6925.92917442, 12188.07840494,
        18823.        ])                                               ]
bin min 336.0
bin 1 2964.952528685514
bin 2 6925.929174424704
bin 3 12188.07840493518
bin max 18823.0


# Binarizer

Binarizer convierte valores numéricos en valores binarios (0 o 1) en función de un umbral.

Se usa cuando quieres transformar una variable numérica en categórica, lo que puede ser útil para mejorar el rendimiento de algunos modelos.

Por ejemplo podemos convertir la variable precio a una variable binaria barato (0) y caro (1) para realizar clasificación binaria.

Otro ejemplo es binarizar la edad de una persona en adulto (0 o 1) en función de si tiene igual o más de 18 años o no.


In [33]:
from sklearn.preprocessing import Binarizer


X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

binarizer = Binarizer(threshold=df['carat'].median())
X_train['carat'] = binarizer.fit_transform(X_train[['carat']])
X_test['carat'] = binarizer.transform(X_test[['carat']])
X_train.head(3)


,carat,depth,table,x,y,z
4227,0.0,63.2,56.0,4.27,4.30,2.71
4676,1.0,65.2,59.0,6.28,6.27,4.09
800,1.0,61.3,58.0,6.10,6.04,3.72


# Imputers 



## SimpleImputer
Clase para imputar valores nulos (NaN) en dataframes o arrays. Estrategias:
mean: Rellena con la media de la columna (numérica).median: Rellena con la mediana de la columna (numérica).most_frequent: Rellena los valores nulos con el valor más frecuente de la columna (puede usarse tanto en numéricas como en categóricas).constant: Rellena con un valor constante que definimos (por ejemplo, "missing" en categóricas o 0 en numéricas).

In [36]:
df.isna().sum() #No existen nulos y vamos a crear nulos o nans

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [ ]:
# Nueva API random recomendable

random_state = np.random.default_rng(seed=42)
df2 = df.copy()
# introducir números nulos aleatoriamente en una columna numérica
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'carat'] = np.nan
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'cut'] = np.nan
df2.isna().sum()

carat      50
cut        50
color       0
clarity     0
depth       0
table       0
price       0
x           0
y           0
z           0
dtype: int64

In [40]:

X = df2[['carat', 'depth', 'table', 'x', 'y', 'z','cut']]
y = df2[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [43]:
from sklearn.impute import SimpleImputer

numerical_cols = X_train.select_dtypes(include=[np.number]).columns
categorical_cols = X_train.select_dtypes(exclude=[np.number]).columns

imputer_num = SimpleImputer(strategy='median')
X_train_numerical = imputer_num.fit_transform(X_train[numerical_cols])
X_test_numerical = imputer_num.transform(X_test[numerical_cols])

# imputer_cat = SimpleImputer(strategy='constant', fill_value='Other')
imputer_cat = SimpleImputer(strategy='most_frequent')
X_train_categorical = imputer_cat.fit_transform(X_train[categorical_cols])
X_test_categorical = imputer_cat.transform(X_test[categorical_cols])

X_train_array = np.concatenate([X_train_numerical, X_train_categorical], axis=1)
X_test_array = np.concatenate([X_test_numerical, X_test_categorical], axis=1)

# opcional: pasar a dataframes de pandas para tener nombres de columnas
X_train_imputed = pd.DataFrame(X_train_array, columns = X_train.columns, index=X_train.index)
X_test_imputed = pd.DataFrame(X_test_array, columns = X_train.columns, index=X_test.index)

print(X_train_imputed.isna().sum())
print(X_test_imputed.isna().sum())

carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64
carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64


## KNNImputer

Imputer que usa algoritmo de K Vecinos más cercanos (K-Nearest Neighbors) para imputar valores faltantes. En lugar de rellenar las celdas nulas (NaN) con una estadística global (como la media o la mediana) de la columna, el KNNImputer busca k instancias (filas) “similares” (más cercanas en el espacio de características) y calcula el valor de la celda faltante a partir de ellas.

In [44]:
random_state = np.random.default_rng(seed=42)
df2 = df.copy()
# introducir números nulos aleatoriamente en una columna numérica
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'carat'] = np.nan
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'cut'] = np.nan
df2.isna().sum()

carat      50
cut        50
color       0
clarity     0
depth       0
table       0
price       0
x           0
y           0
z           0
dtype: int64

In [ ]:
from sklearn.impute import KNNImputer

X = df2[['carat', 'depth', 'table', 'x', 'y', 'z','cut']]
y = df2[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


numerical_cols = X_train.select_dtypes(include=[np.number]).columns
categorical_cols = X_train.select_dtypes(exclude=[np.number]).columns

imputer_num = KNNImputer(n_neighbors=7) # Hemos cambiado SimpleImputers por KNNImputer
X_train_numerical = imputer_num.fit_transform(X_train[numerical_cols])
X_test_numerical = imputer_num.transform(X_test[numerical_cols])


imputer_cat = SimpleImputer(strategy='most_frequent') #Se mantiene SimpleImputer porque KNNIMpuiter no trabaja con categóricas
X_train_categorical = imputer_cat.fit_transform(X_train[categorical_cols])
X_test_categorical = imputer_cat.transform(X_test[categorical_cols])

X_train_array = np.concatenate([X_train_numerical, X_train_categorical], axis=1)
X_test_array = np.concatenate([X_test_numerical, X_test_categorical], axis=1)

# opcional: pasar a dataframes de pandas para tener nombres de columnas
X_train_imputed = pd.DataFrame(X_train_array, columns = X_train.columns, index=X_train.index)
X_test_imputed = pd.DataFrame(X_test_array, columns = X_train.columns, index=X_test.index)

print(X_train_imputed.isna().sum()) #Ya no hay nulos
print(X_test_imputed.isna().sum()) #Ya no hay nulos

carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64
carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64


# IterativeImputer

IterativeImputer implementa una estrategia de imputaciuón multivariable

 A diferencia de los enfoques simples (media, mediana, moda) o KNNImputer, el IterativeImputer entrena un modelo para predecir el valor faltante en cada característica usando las otras características como predictores.

In [47]:
random_state = np.random.default_rng(seed=42)
df2 = df.copy()
# introducir números nulos aleatoriamente en una columna numérica
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'carat'] = np.nan
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'cut'] = np.nan
df2.isna().sum()

carat      50
cut        50
color       0
clarity     0
depth       0
table       0
price       0
x           0
y           0
z           0
dtype: int64

In [53]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [54]:

X = df2[['carat', 'depth', 'table', 'x', 'y', 'z','cut']]
y = df2[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


numerical_cols = X_train.select_dtypes(include=[np.number]).columns
categorical_cols = X_train.select_dtypes(exclude=[np.number]).columns

model = RandomForestRegressor(random_state=42)
imputer_num = IterativeImputer(model, random_state=42, initial_strategy='median') # Hemos cambiado SimpleImputers por IterativeImputer
X_train_numerical = imputer_num.fit_transform(X_train[numerical_cols])
X_test_numerical = imputer_num.transform(X_test[numerical_cols])

model = RandomForestClassifier(random_state=42)
imputer_cat = SimpleImputer(strategy='most_frequent') #Se mantiene SimpleImputer porque KNNIMpuiter no trabaja con categóricas
X_train_categorical = imputer_cat.fit_transform(X_train[categorical_cols])
X_test_categorical = imputer_cat.transform(X_test[categorical_cols])

X_train_array = np.concatenate([X_train_numerical, X_train_categorical], axis=1)
X_test_array = np.concatenate([X_test_numerical, X_test_categorical], axis=1)

# opcional: pasar a dataframes de pandas para tener nombres de columnas
X_train_imputed = pd.DataFrame(X_train_array, columns = X_train.columns, index=X_train.index)
X_test_imputed = pd.DataFrame(X_test_array, columns = X_train.columns, index=X_test.index)

print(X_train_imputed.isna().sum()) #Ya no hay nulos
print(X_test_imputed.isna().sum()) #Ya no hay nulos

carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64
carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64


In [ ]:
# Ejemplo usando IterativeImputer para categóricas (hay que hacer one hot encoder primero)
from sklearn.preprocessing import OrdinalEncoder


df2 = df.copy()
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'carat'] = np.nan
indices = random_state.choice(df2.index, size=50, replace=False)
df2.loc[indices, 'cut'] = np.nan

X = df2[['carat', 'depth', 'table', 'x', 'y', 'z', 'cut']]
y = df2[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

numerical_cols = X_train.select_dtypes(include=[np.number]).columns
categorical_cols = X_train.select_dtypes(exclude=[np.number]).columns

model = RandomForestRegressor(random_state=42)
imputer_num = IterativeImputer(model, random_state=42, initial_strategy='median') # Hemos cambiado a IterativeImputer
X_train_numerical = imputer_num.fit_transform(X_train[numerical_cols])
X_test_numerical = imputer_num.transform(X_test[numerical_cols])

#  En principio InterativeImputer está pensado para columnas numéricas, por lo que si queremos limpiar categóricas

encoder = OrdinalEncoder() # CUIDADO: puede introducir una jerarquia u orden ficticio: 0, 1, 2, 3 ..... puede sugerir una va detrás de otra

X_train_categorical = encoder.fit_transform(X_train[categorical_cols])
X_test_categorical = encoder.transform(X_test[categorical_cols])

model = RandomForestClassifier(random_state=42)
imputer_cat = IterativeImputer(model, random_state=42, initial_strategy='most_frequent') 
X_train_categorical = imputer_cat.fit_transform(X_train_categorical)
X_test_categorical = imputer_cat.transform(X_test_categorical)

X_train_array = np.concatenate([X_train_numerical, X_train_categorical], axis=1)
X_test_array = np.concatenate([X_test_numerical, X_test_categorical], axis=1)

# opcional: pasar a dataframes de pandas para tener nombres de columnas
encoded_columns = encoder.get_feature_names_out(categorical_cols)
all_columns = list(numerical_cols) + list(encoded_columns)

X_train_imputed = pd.DataFrame(X_train_array, columns = all_columns, index=X_train.index)
X_test_imputed = pd.DataFrame(X_test_array, columns = all_columns, index=X_test.index)

print(X_train_imputed.isna().sum()) # Ya no hay nulos
print(X_test_imputed.isna().sum()) # Ya no hay nulos

carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64
carat    0
depth    0
table    0
x        0
y        0
z        0
cut      0
dtype: int64


# Outliers

Los outliers son valores que se encuentran significativamente alejados de la mayoría de los datos en un conjunto de datos. Estos valores pueden deberse a errores en la recopilación, errores de entrada, condiciones extremas o eventos raros. En términos estadísticos, los outliers pueden ser definidos utilizando diferentes métodos, como:
Desviación estándar: Valores que se encuentran más allá de 2 o 3 desviaciones estándar de la media.Rango intercuartílico (IQR): Datos que están fuera del rango [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR], donde IQR es la diferencia entre el tercer y primer cuartil.
Los outliers afectan al modelado:
Distorsionan las métricas estadísticasDificultan la generalizaciónPueden provocar sobreajuste
Algoritmos sensibles a outliers (requieren preprocesamiento):
Regresión lineal y polinómicaKNNSVM con kernel linealRedes neuronalesKMeans y PCA
Algoritmos menos sensibles a outliers (no les afectan tanto):
Regresión logísticaÁrboles de decisiónGradient BoostingAlgoritmos basados en medianas: IsolationForest

In [65]:
X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Filtrar outliers manualmente con Pandas y el método tukey IQR
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 -1.5 *IQR  # LÍMITE INFERIOR
upper_bound = Q1 +1.5 *IQR  # Limite Superior

filtro = ~((X_train < lower_bound) | (X_train > upper_bound)).any(axis=1)
x_train_filter = X_train[filtro]
y_train_filter = y_train[filtro]


filtro = ~((X_test < lower_bound) | (X_test > upper_bound)).any(axis=1)
x_test_filter = X_test[filtro]
y_test_filter = y_test[filtro]

calculate_metrics('Outliers IQR', x_train_filter, x_test_filter, y_train_filter, y_test_filter)

d:\Nueva carpeta\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Nueva carpeta\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
34,RandomForest,OneHotEncoder,0.963541,397.252065,788.302258,0.097110
39,RandomForest,OneHotEncoder+MinMaxscaler,0.963456,397.241947,789.227699,0.097156
38,DecisionTree,OneHotEncoder+MinMaxscaler,0.935301,519.908000,1050.125344,0.126376
33,DecisionTree,OneHotEncoder,0.930697,528.805000,1086.851350,0.126546
35,LinearRegression,OneHotEncoder+MinMaxscaler,0.914293,791.529000,1208.650284,0.439170
30,LinearRegression,OneHotEncoder,0.914045,791.159437,1210.397029,0.438661
31,KNN,OneHotEncoder,0.887188,764.857600,1386.662491,0.201587
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055


# IsolationForest
Algoritmo de conjunto para detectar anómalias o outliers

In [76]:
from sklearn.ensemble import IsolationForest

X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#outliers_detector = IsolationForest(contamination=0.05, random_state=42) # Indicamos se espera un 5% de outliers y si no ponemos nada lo hace de forma automática 
outlier_detector = IsolationForest(contamination='auto', random_state=42) # indicamos que se espera 5 % de outliers, por defecto es auto
outlier_detector.fit(X_train)
# Eliminar datos outliers de train:
y_train_pred = outlier_detector.predict(X_train) # devuelve un array así: array([ 1,  1, -1]) donde -1 es que es anómalo
filtro = (y_train_pred != -1)
X_train_filtered = X_train[filtro]
y_train_filtered = y_train[filtro]
print('X_train len', X_train.shape[0])
print('X_train_filtered len', X_train_filtered.shape[0]) # Si lo dejamos en 'auto' elimina más que el IQR
y_test_pred = outlier_detector.predict(X_test) # no hacemos fit, igual que preprocessors para evitar data leakage
filtro = (y_test_pred != -1)
X_test_filtered = X_test[filtro]
y_test_filtered = y_test[filtro]
print('X_test len', X_test.shape[0])
print('X_test_filtered len', X_test_filtered.shape[0])

# detecta 3279 filas anómalas

X_train len 4000
X_train_filtered len 3279
X_test len 1000
X_test_filtered len 806


In [77]:
calculate_metrics('InsolationForest', x_train_filter, x_test_filter, y_train_filter, y_test_filter )

d:\Nueva carpeta\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Nueva carpeta\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Modelo,Preprocesados,R2,MAE,RMSE,MAPE
34,RandomForest,OneHotEncoder,0.963541,397.252065,788.302258,0.097110
39,RandomForest,OneHotEncoder+MinMaxscaler,0.963456,397.241947,789.227699,0.097156
38,DecisionTree,OneHotEncoder+MinMaxscaler,0.935301,519.908000,1050.125344,0.126376
33,DecisionTree,OneHotEncoder,0.930697,528.805000,1086.851350,0.126546
35,LinearRegression,OneHotEncoder+MinMaxscaler,0.914293,791.529000,1208.650284,0.439170
30,LinearRegression,OneHotEncoder,0.914045,791.159437,1210.397029,0.438661
31,KNN,OneHotEncoder,0.887188,764.857600,1386.662491,0.201587
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
